# 🖼️ DailyFlash Offer Image Generator

This notebook generates promotional images from structured offer JSON using Stable Diffusion models on Google Colab.

## Overview
1. Setup and installation
2. Load Stable Diffusion model
3. Define prompt templates
4. Generate promotional images
5. Save and download images

## Step 1: Setup & Installation

First, let's install the necessary packages and configure Google Drive.

In [ ]:
# Install required libraries
!pip install -q diffusers transformers accelerate ftfy jinja2 pillow

In [ ]:
# Mount Google Drive for saving images
from google.colab import drive
drive.mount('/content/drive')

# Create directory to save images
import os
os.makedirs('/content/drive/MyDrive/offer_images', exist_ok=True)

## Step 2: Load Stable Diffusion Model

We'll use the HuggingFace `diffusers` library to load a Stable Diffusion model. The first time you run this, it will download the model weights (around 4-7 GB).

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import json
import jinja2

# Check if CUDA (GPU) is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the Stable Diffusion model
# You can try different models like:
# - "stabilityai/stable-diffusion-2" (standard quality)
# - "stabilityai/stable-diffusion-2-1" (better quality)
# - "runwayml/stable-diffusion-v1-5" (classic model)
model_id = "stabilityai/stable-diffusion-2-1"

# Load the pipeline
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

# Enable memory efficient attention if needed (for GPUs with limited memory)
pipe.enable_attention_slicing()

## Step 3: Define Prompt Templates

We'll use Jinja2 templates to format our prompts based on the offer JSON.

In [ ]:
from jinja2 import Template

# Define template for promotional posters
# We'll have a few different templates for different types of offers

# General promotional poster template
general_template = """
Create a vibrant promotional poster for a {{ category }} offer.
Title: "{{ title }}"
Subtitle: "{{ subtitle }}"
{% if highlights %}
Highlights:
{% for highlight in highlights %}
- {{ highlight }}
{% endfor %}
{% endif %}
{% if cta %}
Call to Action: "{{ cta }}"
{% endif %}
{% if store_name and location %}
Store: {{ store_name }}, {{ location }} {% if contact %}({{ contact }}){% endif %}
{% endif %}
High quality, professional advertising poster, bright colors, bold fonts, eye-catching design, square format (1:1), suitable for social media, photorealistic
"""

# Fashion sale template
fashion_template = """
Create a vibrant promotional poster for a fashion retail {{ season }} sale.
Title: "{{ title }}"
Subtitle: "{{ subtitle }}"
{% if highlights %}
Highlights:
{% for highlight in highlights %}
{% endfor %}
{% endif %}
{% if cta %}
Call to Action: "{{ cta }}"
{% endif %}
{% if store_name and location %}
Store: {{ store_name }}, {{ location }} {% if contact %}({{ contact }}){% endif %}
{% endif %}
{% if season == "monsoon" %}
Add rain effects, umbrella icons, puddles, and bright colors.
{% elif season == "summer" %}
Add sun rays, beach elements, bright summer colors, and refreshing visuals.
{% elif season == "winter" %}
Add snowflakes, warm clothing icons, cozy atmosphere, and holiday elements.
{% elif season == "festival" or season == "diwali" %}
Add festive lights, diyas, bright celebratory colors, and festive decorations.
{% endif %}
Format as a square (1:1), suitable for Instagram. Include icons for clothes and shopping.
High quality, professional advertising poster, bold fonts, eye-catching design, photorealistic
"""

# Food promotion template
food_template = """
Create a mouth-watering promotional poster for a food offer at a restaurant.
Title: "{{ title }}"
Subtitle: "{{ subtitle }}"
{% if highlights %}
Highlights:
{% for highlight in highlights %}
- {{ highlight }}
{% endfor %}
{% endif %}
{% if cta %}
Call to Action: "{{ cta }}"
{% endif %}
{% if store_name and location %}
Restaurant: {{ store_name }}, {{ location }} {% if contact %}({{ contact }}){% endif %}
{% endif %}
Show delicious food, appetizing colors, steam rising from food, beautiful plating.
Format as a square (1:1), suitable for Instagram. Include food icons and tableware.
High quality, professional food photography style, bright lighting, sharp focus, photorealistic
"""

# Electronics sale template
electronics_template = """
Create a tech-themed promotional poster for an electronics offer.
Title: "{{ title }}"
Subtitle: "{{ subtitle }}"
{% if highlights %}
Highlights:
{% for highlight in highlights %}
- {{ highlight }}
{% endfor %}
{% endif %}
{% if cta %}
Call to Action: "{{ cta }}"
{% endif %}
{% if store_name and location %}
Store: {{ store_name }}, {{ location }} {% if contact %}({{ contact }}){% endif %}
{% endif %}
Modern tech design with digital elements, circuit patterns, glowing edges, and futuristic style.
Format as a square (1:1), suitable for Instagram. Include tech gadget icons and digital elements.
High quality, professional advertising poster, bold fonts, eye-catching design, photorealistic
"""

# Dictionary mapping categories to templates
templates = {
    "clothing": fashion_template,
    "fashion": fashion_template,
    "food": food_template,
    "restaurant": food_template,
    "electronics": electronics_template,
    "tech": electronics_template,
    # Default to general template for other categories
}

## Step 4: Helper Functions for Image Generation

In [ ]:
def prepare_offer_for_prompt(offer_json):
    """
    Prepares an offer JSON for prompt generation by extracting and formatting relevant fields
    """
    # If offer_json is a string, parse it
    if isinstance(offer_json, str):
        offer = json.loads(offer_json)
    else:
        offer = offer_json
    
    # Extract basic info
    title = offer.get('title', '')
    subtitle = offer.get('subtitle', '')
    location = offer.get('location', '')
    contact = offer.get('contact', '')
    category = offer.get('category', '').lower()
    discount_type = offer.get('discount_type', '')
    expiry_date = offer.get('expiry_date', '')
    
    # Determine the store name from the title or set to default
    store_name = title.split(' ')[0] if title else 'Store'
    
    # Create highlights
    highlights = []
    if discount_type == 'percentage' and 'discount_value' in offer:
        highlights.append(f"🔥 {offer['discount_value']}% OFF")
    elif discount_type == 'bundle':
        highlights.append(f"🎁 {subtitle}")
    elif discount_type == 'free_gift' and 'gift_value' in offer:
        highlights.append(f"🎁 Free Gift Worth ₹{offer['gift_value']}")
    
    # Add expiry as a highlight if present
    if expiry_date:
        highlights.append(f"⏰ Offer valid till {expiry_date}")
    
    # Add promotional code if present
    if 'promo_code' in offer:
        highlights.append(f"🏷️ Use Code: {offer['promo_code']}")
    
    # Determine season from title or category (for fashion template)
    season = "regular"
    title_lower = title.lower()
    if any(s in title_lower for s in ["monsoon", "rain"]):
        season = "monsoon"
    elif any(s in title_lower for s in ["summer", "hot"]):
        season = "summer"
    elif any(s in title_lower for s in ["winter", "cold"]):
        season = "winter"
    elif any(s in title_lower for s in ["diwali", "festival", "celebration"]):
        season = "festival"
    
    # Default call to action
    cta = "Shop Now!"
    if expiry_date:
        cta = "Hurry! Limited Time Offer"
    
    return {
        "title": title,
        "subtitle": subtitle,
        "highlights": highlights,
        "cta": cta,
        "store_name": store_name,
        "location": location,
        "contact": contact,
        "category": category,
        "season": season
    }

def generate_prompt_from_offer(offer_json):
    """
    Generate a prompt for the image generation model from an offer JSON
    """
    # Prepare data for template
    data = prepare_offer_for_prompt(offer_json)
    
    # Choose template based on category
    category = data.get('category', '').lower()
    template_str = templates.get(category, general_template)
    
    # Render template with data
    template = Template(template_str)
    prompt = template.render(**data)
    
    # Clean up whitespace and newlines
    prompt = ' '.join(prompt.split())
    
    return prompt

def generate_offer_image(offer_json, num_inference_steps=50, guidance_scale=7.5, seed=None):
    """
    Generate an image from an offer JSON
    """
    # Generate prompt from offer
    prompt = generate_prompt_from_offer(offer_json)
    print(f"Generated prompt: {prompt}")
    
    # Set random seed for reproducibility if provided
    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)
    else:
        generator = None
    
    # Generate the image
    image = pipe(
        prompt,
        height=512,
        width=512,  # Square format for Instagram
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images[0]
    
    return image, prompt

def save_image(image, filename, directory="/content/drive/MyDrive/offer_images"):
    """
    Save the generated image to the specified directory
    """
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    image.save(path)
    print(f"Image saved to {path}")
    return path

## Step 5: Test with Example Offers

In [ ]:
# Example offer JSON
example_offer = {
    "title": "ABC Store Super Shirt Sale!",
    "subtitle": "Buy 2 Get 1 Free",
    "location": "Lucknow",
    "contact": "+919876543210",
    "category": "Clothing",
    "discount_type": "bundle",
    "expiry_date": "2025-08-31"
}

# Generate image
image, prompt = generate_offer_image(example_offer)

# Display the image
display(image)

# Save the image
save_image(image, "abc_store_sale.png")

## Step 6: Function to Process New Offers

In [ ]:
def process_new_offer(offer_json_str):
    """
    Process a new offer from JSON string and generate an image
    """
    try:
        # Parse JSON if it's a string
        offer = json.loads(offer_json_str) if isinstance(offer_json_str, str) else offer_json_str
        
        # Generate a clean filename from the title
        title_slug = offer.get("title", "offer").lower().replace(" ", "_")[:30]
        filename = f"{title_slug}_{int(time.time())}.png"
        
        # Generate and save image
        image, prompt = generate_offer_image(offer)
        path = save_image(image, filename)
        
        # Display the image
        display(image)
        
        return {
            "success": True,
            "image_path": path,
            "prompt": prompt
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

# Test with a new offer
import time

new_offer = {
    "title": "FoodDelight Weekend Special",
    "subtitle": "50% Off All Pizzas",
    "location": "Mumbai",
    "contact": "1800-123-4567",
    "category": "Food",
    "discount_type": "percentage",
    "discount_value": 50,
    "expiry_date": "weekend"
}

result = process_new_offer(new_offer)
print(result)

## Step 7: Interactive Input Form

In [ ]:
# Create an interactive form for entering offer JSON
from ipywidgets import widgets
from IPython.display import display, clear_output

text_area = widgets.Textarea(
    value='{
    "title": "DCOT Monsoon Mega Sale!",
    "subtitle": "Unbeatable Offers at DCOT by DONER",
    "location": "Faizabad Road",
    "contact": "+91 9208619124",
    "category": "Clothing",
    "discount_type": "bundle",
    "promo_code": "MONSOON25",
    "expiry_date": "2025-08-31"
}',
    placeholder='Enter offer JSON here',
    description='Offer JSON:',
    disabled=False,
    layout=widgets.Layout(height='200px', width='100%')
)

button = widgets.Button(
    description='Generate Offer Image',
    disabled=False,
    button_style='success',
    tooltip='Click to generate image',
    icon='check'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        try:
            print("Generating offer image...")
            result = process_new_offer(text_area.value)
            if result["success"]:
                print(f"Image saved to: {result['image_path']}")
                print(f"Prompt used: {result['prompt']}")
            else:
                print(f"Error: {result['error']}")
        except Exception as e:
            print(f"Error: {str(e)}")

button.on_click(on_button_clicked)

display(text_area)
display(button)
display(output)

## Download All Generated Images

In [ ]:
# Zip all images for download
!zip -r /content/offer_images.zip /content/drive/MyDrive/offer_images
from google.colab import files
files.download('/content/offer_images.zip')

# 🎉 Congratulations!

You've successfully created an offer image generator using Stable Diffusion. This complements your text-based offer generator, providing a complete solution for creating promotional content.

## Next Steps

1. Integrate this with your text-based offer generator
2. Experiment with different templates and prompts
3. Fine-tune the generation parameters for better results
4. Create a unified interface for both text and image generation